In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tqdm import tqdm

from attacks import GM
from classifiers import CIFAR_CNN
from trainers import Trainer
from utils import params_maker
from utils import unpickle

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

datadir = './CIFAR10_data/'
batches = [datadir + batch for batch in os.listdir(datadir)]


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
for i in tqdm(range(5)):
    batch = unpickle(batches[i])

    if i == 0:
        data = batch[b'data'].astype(np.float32)
        cifar = np.transpose(np.reshape(data, [-1, 3, 32, 32]), [0, 2, 3, 1])
        labels = batch[b'labels']
    else:
        data = batch[b'data'].astype(np.float32)
        cifar = np.concatenate((cifar, np.transpose(np.reshape(data, [-1, 3, 32, 32]), [0, 2, 3, 1])), axis=0)
        labels = np.concatenate((labels, batch[b'labels']), axis=0)

scaled_cifar = cifar / 127.5 - 1.0

test_batch = unpickle(batches[5])
cifar_test = np.transpose(np.reshape(test_batch[b'data'], [-1, 3, 32, 32]), [0, 2, 3, 1])
scaled_cifar_test = cifar_test / 127.5 - 1.0
labels_test = np.array(test_batch[b'labels'])

data_train = (scaled_cifar, labels)
data_test = (scaled_cifar_test, labels_test)


100%|██████████| 5/5 [00:02<00:00,  2.49it/s]


In [4]:
use_batchnorm = False
droprate = None
snbeta = None
l2rate = None

attack = GM

attack_paramss = params_maker(['eps', 'step_size', 'n_steps', 'norm', 'loss_type'], \
                              [[0.005], [0.02], [40], ['inf'], ['xent', 'cw']])

for attack_params in attack_paramss:

    logdir = 'tf_logs/exp3/'
    if use_batchnorm: logdir += 'batchnorm/'
    if droprate: logdir += 'dropout/{}/'.format(droprate)
    if snbeta: logdir += 'sn/{}/'.format(snbeta)
    if l2rate: logdir += 'l2/{}/'.format(l2rate)

    if attack: logdir += 'adv/{}/{}/'.format(attack.__name__, '_'.join([str(v) for k, v in attack_params]))

    if logdir.split('/')[-2] == 'exp3': logdir += 'standard/'

    print(logdir)

    tf.reset_default_graph()

    DNN = CIFAR_CNN(logdir, tf.nn.relu, use_batchnorm, droprate, snbeta, l2rate, attack, dict(attack_params))

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    trainer = Trainer(sess, DNN, data_train)
    trainer.train(n_epochs=20, p_epochs=5)

    sess.close()

    # Test Accuracy
    tf.reset_default_graph()

    sess = tf.InteractiveSession()

    DNN = CIFAR_CNN(logdir, tf.nn.relu)
    DNN.load(sess)

    print('Test Accuracy : {:.5f}'.format(DNN.evaluate(sess, data_test)))

    sess.close()


tf_logs/exp4/adv/GM/0.005_0.02_40_inf_xent/
Epoch : 5   | Loss : 0.56201 | Accuracy : 0.79646 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 10  | Loss : 0.18490 | Accuracy : 0.93330 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 15  | Loss : 0.09240 | Accuracy : 0.96756 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 20  | Loss : 0.06591 | Accuracy : 0.97830 | Test Accuracy : nan | Adv Accuracy : nan
INFO:tensorflow:Restoring parameters from tf_logs/exp4/adv/GM/0.005_0.02_40_inf_xent/model
Test Accuracy : 0.71230
tf_logs/exp4/adv/GM/0.005_0.02_40_inf_cw/
Epoch : 5   | Loss : 0.54915 | Accuracy : 0.79394 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 10  | Loss : 0.18371 | Accuracy : 0.93154 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 15  | Loss : 0.07757 | Accuracy : 0.97248 | Test Accuracy : nan | Adv Accuracy : nan
Epoch : 20  | Loss : 0.06587 | Accuracy : 0.97780 | Test Accuracy : nan | Adv Accuracy : nan
INFO:tensorflow:Restoring parameters from tf_logs/exp4/